In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import csv

app = dash.Dash(__name__)
server = app.server
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

df1 = pd.read_csv('nama_10_gdp_1_Data.csv')
df = df1[~df1.GEO.str.contains("Euro")]   #dropping all the rows that don't contain a country in the column GEO
df = df[(df.UNIT == 'Current prices, million euro')]
df.replace(':', np.nan)

#indicators for 1st graph
available_indicators = df['NA_ITEM'].unique()

#indicators for 2nd graph
item_indicators = df['NA_ITEM'].unique()
countries = df['GEO'].unique()



app.layout = html.Div([
    html.Div([
        html.H1(children='1st Graph: select two indicators'),
        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Gross domestic product at market prices'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}), 

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Value added, gross'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['TIME'].min(),
        max=df['TIME'].max(),
        value=df['TIME'].max(),
        step=None,
        marks={str(year): str(year) for year in df['TIME'].unique()}
    ),

    html.H1(children='2nd Graph: select country and an indicator'),
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='country',
                options=[{'label': i, 'value': i} for i in countries],
                value='Belgium'
            )],
        style={'width': '48%', 'display': 'inline-block'}), #this only takes 50% of the webpage

        
        html.Div([
            dcc.Dropdown(
                id='yaxis-column1',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Value added, gross'
            )],
        style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic1')
])

@app.callback(   
    dash.dependencies.Output('indicator-graphic', 'figure'), 
    [dash.dependencies.Input('xaxis-column', 'value'),
     dash.dependencies.Input('yaxis-column', 'value'),
     dash.dependencies.Input('xaxis-type', 'value'),
     dash.dependencies.Input('yaxis-type', 'value'),
     dash.dependencies.Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['TIME'] == year_value]  #filtering data by year value
    
    return {
        'data': [go.Scatter(
            x=dff[dff['NA_ITEM'] == xaxis_column_name]['Value'],
            y=dff[dff['NA_ITEM'] == yaxis_column_name]['Value'],
            text=dff[dff['NA_ITEM'] == yaxis_column_name]['GEO'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 60, 'b': 40, 't': 10, 'r': 60},
            hovermode='closest'
        )
    }
@app.callback(   
    dash.dependencies.Output('indicator-graphic1', 'figure'),  
    [dash.dependencies.Input('country', 'value'),
     dash.dependencies.Input('yaxis-column1', 'value')])

def update_graph(country_name, yaxis_column_name):
    data_country = df[df['GEO'] == country_name]
    
    return {
        'data': [go.Scatter(
            x=data_country[data_country['NA_ITEM'] == yaxis_column_name]['TIME'],
            y=data_country[data_country['NA_ITEM'] == yaxis_column_name]['Value'],
            text=data_country[data_country['NA_ITEM'] == yaxis_column_name]['Value'],
            mode='lines',
        )],
        'layout': go.Layout(
            xaxis={
                'title':"YEAR",
                'type':'linear'},
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear'},
            margin={'l': 60, 'b': 40, 't': 10, 'r': 60},
            hovermode='closest'
        )
    }

    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2018 12:51:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2018 12:52:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2018 12:52:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2018 12:52:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2018 12:52:06] "POST /_dash-update-component HTTP/1.1" 200 -
